# Chapter 5

<a target="_blank" href="https://colab.research.google.com/github/wandb/edu/blob/main/rag-advanced/notebooks/Chapter05.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<!--- @wandbcode{rag-course-05} -->

## Retrieval and Re-ranking

In this chapter, we will focus on improving the retrieval quality and employ reranking to select the best context for our LLM.

In the last chapter, we added Query Enhancement module which was able to derive sub-queries from the user query. We retrieved contexts for these sub-queries. We did a basic deduplication of the total context to reduce the same context. However on a semantic level there might be few more contexts that we can drop thus reducing the input token cost.

When the same context is retrieved a few times, we can use this knowledge to fuse the confidence scores of such contexts and ask our LLM to focus more on them. Once can employ many strategies to improve the quality of contexts we provide our LLM.

Let's see a few here.

In [1]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone https://github.com/wandb/edu.git
    %cd edu/rag-advanced
    !pip install -qqq -r requirements.txt
    %cd notebooks
else:
    print("Not running in Google Colab. Skipping git clone and pip install commands.")

Not running in Google Colab. Skipping git clone and pip install commands.


In [2]:
import nltk

nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anishshah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/anishshah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from set_env import set_env

set_env("WANDB_API_KEY")
set_env("OPENAI_API_KEY")
set_env("COHERE_API_KEY")

2024-10-13 23:26:33.561 | WARNING  | set_env.set_env:set_env:109 - 
        Unable to set WANDB_API_KEY=WANDB_API_KEY,
        not in colab or Secrets not set, not kaggle
        or Secrets not set, no .env/dotenv/env file
        in the current working dir or parent dirs.


'c5bu7QSN1JNxiVuMUpwJjpWN7jqPBBK6TV2EgSrC'

In [4]:
import asyncio

import nest_asyncio

nest_asyncio.apply()

import weave

In [5]:
WANDB_PROJECT = "rag-course-finance"

weave_client = weave.init(WANDB_PROJECT)

There are different credentials in the netrc file and the environment. Using the environment value.


Logged in as Weights & Biases user: a-sh0ts.
View Weave data at https://wandb.ai/a-sh0ts/rag-course-finance/weave


We will download the chunked data from chapter 3

In [6]:
# Reload the data from Chapter 3
chunked_data = weave.ref(
    "chunked_data:latest"
).get()
# uncomment the next line to get the chunked data from weave from your own project instead
# chunked_data = weave.ref("chunked_data:latest").get()

chunked_data.rows[:2]
chunked_data = list(map(dict, chunked_data.rows[:]))

## Embedding based retriever

In the past chapters we were using TF-IDF and BM25 based retrievers. The vector representation from these methods are not "dense" i.e, they were not trained on billions or trillions of tokens. LLMs today embed the tokens into a dense representation. These embeddings are learned during the training process and are used to represent the tokens in the high-dimensional vector space. They have a denser knowledge of different lingustic patterns.

One stratightforward way of improving our retriever is to use a `DenseRetriever` which uses the embedding model (Cohere embedding model here) to embed the chunks and uses the same embedding model to embed the query.

## Reranking the contexts

With more contexts it is important to pick the one that adds more knowledge about the given query. For this, a re-ranking model is used that calculates a matching score for a given query and document pair. This score can then be utilized to rearrange vector search results, ensuring that the most relevant results are prioritized at the top of the list. Cohere comes with it's own re-ranking model and is quite popular.


In [7]:
from scripts.reranker import LiteLLMReranker
from scripts.retriever import DenseRetriever, DenseRetrieverWithReranker
from scripts.utils import display_source

display_source(DenseRetriever)
display_source(LiteLLMReranker)
display_source(DenseRetrieverWithReranker)

   1 class DenseRetriever(weave.Model):                                                                            
   2     """                                                                                                       
   3     A retriever model that uses dense embeddings for indexing and searching documents.                        
   4                                                                                                               
   5     Attributes:                                                                                               
   6         vectorizer (Callable): The function used to generate embeddings.                                      
   7         index (np.ndarray): The indexed embeddings.                                                           
   8         data (list): The data to be indexed.                                                                  
   9     """                                                                                                       
  10                                                                                                               
  11     vectorizer: Callable = sync_embed                                                                         
  12     index: np.ndarray = None                                                                                  
  13     data: list = None                                                                                         
  14                                                                                                               
  15     def index_data(self, data):                                                                               
  16         """                                                                                                   
  17         Indexes the provided data using dense embeddings.                                                     
  18                                                                                                               
  19         Args:                                                                                                 
  20             data (list): A list of documents to be indexed. Each document should be a dictionary              
  21                          containing a key 'cleaned_content' with the text to be indexed.                      
  22         """                                                                                                   
  23         self.data = data                                                                                      
  24         docs = [doc["cleaned_content"] for doc in data]                                                       
  25         embeddings = self.vectorizer(docs)                                                                    
  26         self.index = np.array(embeddings)                                                                     
  27                                                                                                               
  28     @weave.op()                                                                                               
  29     def search(self, query, k=5):                                                                             
  30         """                                                                                                   
  31         Searches the indexed data for the given query using cosine similarity.                                
  32                                                                                                               
  33         Args:                                                                                                 
  34             query (str): The search query.                                                                    
  35             k (int): The number of top results to r

   1 class LiteLLMReranker(weave.Model):                                                                           
   2     """                                                                                                       
   3     A class to rerank documents using LiteLLM's reranking capabilities.                                       
   4     """                                                                                                       
   5                                                                                                               
   6     model: str = "cohere/rerank-english-v3.0"                                                                 
   7                                                                                                               
   8     @weave.op()                                                                                               
   9     def rerank(self, query, docs, top_n=None):                                                                
  10         """                                                                                                   
  11         Reranks the given documents based on their relevance to the query.                                    
  12                                                                                                               
  13         Args:                                                                                                 
  14             query (str): The query string.                                                                    
  15             docs (List[Dict[str, Any]]): A list of documents to be reranked.                                  
  16             top_n (int, optional): The number of top documents to return. Defaults to None.                   
  17                                                                                                               
  18         Returns:                                                                                              
  19             List[Dict[str, Any]]: A list of reranked documents with relevance scores.                         
  20         """                                                                                                   
  21         documents = [doc["text"] for doc in docs]                                                             
  22         response = rerank(                                                                                    
  23             model=self.model,                                                                                 
  24             query=query,                                                                                      
  25             documents=documents,                                                                              
  26             top_n=top_n or len(docs)                                                                          
  27         )                                                                                                     
  28                                                                                                               
  29         outputs = []                                                                                          
  30         for result in response:                                                                               
  31             reranked_doc = docs[result['index']]                                                              
  32             reranked_doc["relevance_score"] = result['relevance_score']                                       
  33             outputs.append(reranked_doc)                                                                      
  34         return outputs[:top_n]                                                                                
  35                                                    

   1 class DenseRetrieverWithReranker(weave.Model):                                                                
   2     """                                                                                                       
   3     A retriever model that uses dense embeddings for retrieval and a reranker for re-ranking the results.     
   4                                                                                                               
   5     Attributes:                                                                                               
   6         retriever (DenseRetriever): The dense retriever model.                                                
   7         reranker (LiteLLMReranker): The reranker model.                                                       
   8     """                                                                                                       
   9                                                                                                               
  10     retriever: DenseRetriever = DenseRetriever()                                                              
  11     reranker: LiteLLMReranker = LiteLLMReranker()                                                             
  12                                                                                                               
  13     def index_data(self, data):                                                                               
  14         """                                                                                                   
  15         Indexes the provided data using the dense retriever.                                                  
  16                                                                                                               
  17         Args:                                                                                                 
  18             data (list): A list of documents to be indexed. Each document should be a dictionary              
  19                          containing a key 'cleaned_content' with the text to be indexed.                      
  20         """                                                                                                   
  21         self.retriever.index_data(data)                                                                       
  22                                                                                                               
  23     @weave.op()                                                                                               
  24     def predict(self, query: str, top_k: int = None, top_n: int = None):                                      
  25         """                                                                                                   
  26         Predicts the top-n results for the given query after re-ranking.                                      
  27                                                                                                               
  28         Args:                                                                                                 
  29             query (str): The search query.                                                                    
  30             top_k (int, optional): The number of top results to retrieve before re-ranking. Default is None.  
  31             top_n (int, optional): The number of top results to return after re-ranking. Default is None.     
  32                                                                                                               
  33         Returns:                                                                                              
  34             list: A list of dictionaries containing the source, text, and score of the top-n results.         
  35         """                                        

Let's initialize the `DenseRetriever` and index the data.

In [8]:
dense_retriever = DenseRetriever()
dense_retriever.index_data(chunked_data)


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/provide

APIConnectionError: litellm.APIConnectionError: CohereException - {"message":"trial token rate limit exceeded, limit is 100000 tokens per minute"}


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers




Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/provide

In [ ]:
from scripts.retrieval_metrics import IR_METRICS

eval_dataset = weave.ref(
    "eval_data:latest"
).get()

retrieval_evaluation = weave.Evaluation(
    name="Dense Retrieval Evaluation",
    dataset=eval_dataset,
    scorers=IR_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "k": 10},
)

dense_retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(dense_retriever))

Let's initialize the `DenseRetrieverWithReranker` and index the data.

In [ ]:
dense_retriever_rerank = DenseRetrieverWithReranker()
dense_retriever_rerank.index_data(chunked_data)

In [ ]:
from scripts.retrieval_metrics import IR_METRICS

eval_dataset = weave.ref(
    "eval_data:latest"
).get()

retrieval_evaluation = weave.Evaluation(
    name="Dense Retrieval Rerank Evaluation",
    dataset=eval_dataset,
    scorers=IR_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "top_k": 20, "top_n": 10},
)

dense_retrieval_scores = asyncio.run(
    retrieval_evaluation.evaluate(dense_retriever_rerank)
)

## Hybrid Retriever

Even though BM25 is an old model used for retrieval tasks, it is still the state-of-the-art on various benchmark. In machine learning, we ensemble a few weak classifiers to build a stronger classifier, we can adopt the same idea to our retriever pipeline.

Below we show the concept of hybrid retriever which uses two or more retrievers and retrievr chunks from all of them followed by re-ranking.

### Fusion reranking

Since the indiviual retrievers are gonna retrieve the same chunks most of the time, simple re-ranking will not be beneficial since the same chunk from each retriever will have similar score, thus the top_k after re-ranking will have more or less the same context.

Instead, we iterate through all the chunks and fuse the score of similar chunk. Finally we sort on the basis of this fused score and return top_k chunks.

In [ ]:
from scripts.retriever import HybridRetrieverReranker

display_source(HybridRetrieverReranker)

hybrid_retriever = HybridRetrieverReranker()

In [ ]:
hybrid_retriever.index_data(chunked_data)

In [ ]:
eval_dataset = weave.ref(
    "eval_data:latest"
).get()

retrieval_evaluation = weave.Evaluation(
    name="Dense Retrieval Rerank Evaluation",
    dataset=eval_dataset,
    scorers=IR_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "top_k": 20, "top_n": 10},
)

hybrid_retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(hybrid_retriever))